In [1]:
import os
import torch
import clip
import requests
from io import BytesIO
from PIL import Image
from supabase import create_client, Client
from dotenv import load_dotenv


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/safiaboutaleb/Developer/eco-wardrobe/test-env/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/safiaboutaleb/Developer/eco-wardrobe/test-env/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/safiaboutaleb/Developer/eco-wardrobe/test-env/lib/python3.11/site-package

In [3]:
# load environment variables - getting secret key and url
print("Script Started") #test print to check if the script runs
load_dotenv()

SUPABASE_URL = os.getenv("VITE_SUPABASE_URL")
SUPABASE_SECRET_KEY = os.getenv("SUPABASE_SECRET_KEY")  # must use service key

if not SUPABASE_URL or not SUPABASE_SECRET_KEY:
    raise ValueError("Missing SUPABASE_URL or SUPABASE_SECRET_KEY in .env")

# create supabase client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_SECRET_KEY)

Script Started


In [5]:
# --- Load CLIP ---
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def compute_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    image = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model.encode_image(image)
    embedding = embedding / embedding.norm(dim=-1, keepdim=True)
    return embedding.cpu().numpy().tolist()[0]

def find_top_matches(image_path, k=3):
    query_embedding = compute_embedding(image_path)

    response = supabase.rpc(
        "match_embeddings",  # stored function name
        {"query_embedding": query_embedding, "match_count": k}
    ).execute()

    return response.data

# --- Run it ---
results = find_top_matches("pink-sweater.png")

for item in results:
    print(item["clothing_name"], item["image_url"], item["distance"])

RuntimeError: Numpy is not available